In [101]:
import numpy as np
import os
import cv2
import category_encoders as ce
import matplotlib.pyplot as plt
from matplotlib.colors import NoNorm
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras import datasets, layers, models
import base64

In [46]:
def LoadData(dataDir,new_size=None):
    if not new_size is None:
        img_rows, img_cols = new_size
    classes=[]
    for _,dirs,_ in os.walk(dataDir):
        classes=dirs
        break
    num_classes=len(classes)
    X_tr=[]
    Y_tr=[]  
    for idx,cl in enumerate(classes):
        print(cl)
        for _,_,files in os.walk(dataDir+"\\" + cl):               
            l=len(files)
            for f in files:
                img_path=dataDir+"\\"+cl+'\\'+f
                img=cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#                 img = cv2.resize(src = img, dsize = None, fx = 0.5, fy = 0.5)
                X_tr.append(img)
                Y_tr.append(cl)
    X_tr=np.array(X_tr)
    Y_tr=np.array(Y_tr)
    return X_tr, Y_tr, classes

In [68]:
def predictBase64(listString, path):
    answer = []
    classes=[]
    imgPredict = []
    for _,dirs,_ in os.walk(path + "\\data"):
        classes=dirs
        break
    classes = sorted(classes)
    for i in listString:
        binaryImg = i.encode('ascii')
        with open("imagePredict.png", "wb") as fh:
            fh.write(base64.decodebytes(binaryImg))
        imgPre = cv2.imread("imagePredict.png")
        imgPre = cv2.cvtColor(imgpre, cv2.COLOR_BGR2RGB)
        imgPredict.append(imgPre)
    model = models.load_model(path + "\\model")
    imgPredict = np.array(imgPredict)
    encodePre = np.argmax(model.predict(imgPredict), axis=-1)
    for i in encodePre:
        answer.append(classes[i]) 
    return answer


In [96]:
chechstring = "..."

In [97]:
listString = []
listString.append(chechstring)

In [98]:
answer = predictBase64(listString, os.getcwd())
answer

1/1 [==============================] - 0s 104ms/step


['chech']

In [105]:
def predictListImage(listImage, path):
    answer = []
    classes=[]
    imgPredict = []
    for _,dirs,_ in os.walk(path + "\\data"):
        classes=dirs
        break
    classes = sorted(classes)
    model = models.load_model(path + "\\model")
    encodePre = np.argmax(model.predict(listImage), axis=-1)
    for i in encodePre:
        answer.append(classes[i]) 
    return answer

In [104]:
trainImage, label, classes = LoadData(os.getcwd() +  "\\data")

chech
goc
te


In [106]:
answer2 = predictListImage(trainImage, os.getcwd())

2/2 [==============================] - 2s 427ms/step


In [85]:
def trainModel(path):
    train_image, train_label, classes = LoadData(path + "\\data")
    idxs=np.random.permutation(len(train_image))
    plt.figure(figsize = (10,10))
    for i in range(12):  # Lấy ngẫu nhiên 12 mẫu trong tập train
        idx=idxs[i]
        plt.subplot(3,4,i+1)
        plt.imshow(train_image[idx],norm=NoNorm())
        plt.title(train_label[idx])
    plt.show()
    le = preprocessing.LabelEncoder()
    le.fit(classes)
    train_label = le.transform(train_label)
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=train_image[0].shape) )
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Dropout(0.25))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(len(classes)))
    model.summary()
    model.compile(optimizer='nadam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
#     history = model.fit(train_image, train_label, epochs=30)
#     model.save(path + "\\save_model")